# Proyecto Dataset Diabético

## 📊 Resumen del Dataset

- **Dimensiones**: 101,766 registros × 50 columnas
- **Variable objetivo**: `readmitted` (NO, <30 días, >30 días)
- **Tipo de problema**: Clasificación multiclase (puede convertirse a binaria)

---


In [8]:
import pandas as pd
import numpy as np
import warnings
from sklearn.impute import SimpleImputer
warnings.filterwarnings('ignore')

## Limpieza y Preprocesamiento de Datos

In [14]:
df = pd.read_csv("data/diabetic_data.csv")
    
print(f"\n✓ Dataset cargado exitosamente")
print(f"  Dimensiones originales: {df.shape[0]} filas × {df.shape[1]} columnas")


✓ Dataset cargado exitosamente
  Dimensiones originales: 101766 filas × 50 columnas


### Eliminación de columnas con mayoría de nulos

Dado que la columna ``weight`` tiene un 97% de nulos, consideramos que la mejor estrategia es eliminarla.

In [15]:
# Drop columna 'weight'
df = df.drop(columns=['weight'])
print(f"  Dimensiones después de eliminar 'weight': {df.shape[0]} filas × {df.shape[1]} columnas")

  Dimensiones después de eliminar 'weight': 101766 filas × 49 columnas


### Imputación de Nulos Menores

**Imputación de ``race`` con la moda**

In [22]:
# Paso 1: Convertir '?' a NaN solo en la columna 'race'
df['race'] = df['race'].replace('?', np.nan)


In [23]:
# Paso 2: Backup y verificación
print("Valores nulos originales en 'race':", df['race'].isna().sum())
df['race_backup'] = df['race'].copy()

Valores nulos originales en 'race': 2273


In [24]:
# Paso 3: Imputación
race_imputer = SimpleImputer(strategy='most_frequent')
df['race'] = race_imputer.fit_transform(df[['race']]).ravel()

In [25]:
# Paso 4: Identificar valores imputados
imputed_mask = df['race_backup'].isna() & df['race'].notna()
imputed_values = df.loc[imputed_mask, 'race']

print(f"\nSe imputaron {len(imputed_values)} valores:")
print(imputed_values.value_counts())


Se imputaron 2273 valores:
race
Caucasian    2273
Name: count, dtype: int64


### Sustitución de nulos en ``payer_code`` y ``medical_specialty`` por ``unknown``

### Transformación de Variable Objetivo

In [6]:
# Opción 1: Readmitido vs No readmitido ¿?
df['readmitted_binary'] = (df['readmitted'] != 'NO').astype(int)

# Opción 2: Readmitido en <30 días vs resto  ¿?
df['early_readmission'] = (df['readmitted'] == '<30').astype(int)

### Standard Scaler para variables numéricas

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_cols = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 
                  'num_medications', 'number_outpatient', 'number_emergency', 
                  'number_inpatient', 'number_diagnoses']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])           

### Balanceo con SMOTE o class_weight = 'balanced'

In [9]:
df = df.drop(['readmitted'], axis=1)  # Eliminar la columna original
print(f"  Dimensiones después de limpieza: {df.shape[0]} filas × {df.shape[1]} columnas\n")
df.to_csv("diabetic_data_cleaned.csv", index=False) # TODO: guarda estos datos dentro de data/

# TODO: quiero ver una vista previa de cómo se verían los nuevos datos balanceados.
print(df.head())

#Plot de los datps balanceados
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6,4))
sns.countplot(x='early_readmission', data=df)
plt.title('Distribución de Readmisiones Tempranas')
plt.xlabel('Readmisión Temprana (<30 días)')
plt.ylabel('Cantidad de Pacientes')
plt.show()



  Dimensiones después de limpieza: 101766 filas × 49 columnas

